In [5]:
%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

from cq_gears import SpurGear, RingGear

def rotate_xy(x, y, angle):
    rx = x * np.cos(angle) - y * np.sin(angle)
    ry = x * np.sin(angle) + y * np.cos(angle)
    return rx, ry

## Ring Gear profile

In [ ]:
module = 1.0
n_teeth = 19
pressure_angle = 20.0

fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')

gear = RingGear(module, n_teeth, width=5.0, rim_width=1.5,
                pressure_angle=pressure_angle)
x, y = gear.gear_points_xy()
ax.plot(x, y, label='gear profile', color='C0')

circles = (
    (gear.r0, 'pitch circle'),
    (gear.ra, 'addendum circle'),
    (gear.rb, 'base circle'),
    (gear.rd, 'dedendum circle'),
)

t = np.linspace(0.0, np.pi * 2.0, 200)

for r, lbl in circles:
    cx = np.cos(t) * r
    cy = np.sin(t) * r
    ax.plot(cx, cy, label=lbl, linestyle='--', linewidth=0.8)

cx = np.cos(t) * gear.rim_r
cy = np.sin(t) * gear.rim_r
ax.plot(cx, cy, color='C0')

plt.legend(loc='center')
plt.show()

## Planetary Gearset

In [9]:
animate = False # Set to True to animate

module = 1.0
sun_teeth = 32
planet_teeth = 16
ring_teeth = sun_teeth + planet_teeth * 2
n_planets = 4

ring = RingGear(module, ring_teeth, 5.0, 5.0)
sun = SpurGear(module, sun_teeth, 5.0)
planet = SpurGear(module, planet_teeth, 5.0)

orbit_r = sun.r0 + planet.r0
planet_a = np.pi * 2.0 / n_planets

ratio = 1.0 / ((planet.z / sun.z) * 2.0)
c_ratio = 1.0 / (ring.z / sun.z + 1.0)

fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')

def plot_gears(frame):
    ax.clear()

    x, y = ring.gear_points_xy()
    ax.plot(x, y)

    w = np.pi * 2.0 / 200.0 * frame
    w1 = w * ratio
    w2 = w * c_ratio
    
    x, y = sun.gear_points_xy()
    x, y = rotate_xy(x, y, sun.tau / 2.0 + w)
    ax.plot(x, y)
    
    for i in range(n_planets):
        x, y = planet.gear_points_xy()
        x, y = rotate_xy(x, y, -w1)
        cx = np.cos(i * planet_a + w2) * orbit_r
        cy = np.sin(i * planet_a + w2) * orbit_r
        x += cx
        y += cy
        ax.plot(x, y, color='C2')


if animate:
    anim = animation.FuncAnimation(fig, plot_gears, 400, interval=25)    
else:
    plot_gears(0)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …